# Experiment Tracking

Purpose of this notebook is to document the good practices that I found from studying the following git page:

Ref Link: https://particle1331.github.io/ok-transformer/nb/mlops/03-mlflow.html#hpo-with-xgboost

First, we need to run mlflow in the background in order to track the logs for our training. To do this, please use the following command shown here:

```bash
mlflow server -h 127.0.0.1 -p 5001 --backend-store-uri=sqlite:///mlflow.db --default-artifact-root=./artifacts
```

SQLite is used to store logs internally in our local file storage, while ./artifacts is used to store the artifacts from mlflow.

I'll experiment with local stack next time to configure an S3 based artifact, so I can study local stack and integrate with mlflow (2 birds 1 stone).

The following displays the project structure I am following for experiment-tracking:

In [5]:
!tree --prune -I 'venv'

.
├── config
│   ├── __init__.py
│   └── core.py
├── data
│   ├── green_tripdata_2021-01.parquet
│   └── green_tripdata_2021-02.parquet
├── model_linear.py
├── model_linear_pudo.py
├── model_trees.py
├── notebook.ipynb
├── properties
│   └── config.yaml
├── requirements.txt
└── utils
    ├── __init__.py
    ├── df_process.py
    ├── df_util.py
    └── feature_util.py

4 directories, 14 files


`config` holds the config class which reads the config yaml from `properties` dir

`data` holds the raw data for this project

`utils` holds the util classes for model classes

The following lines will show the source code of the important classes used. Starting with `df_*` classes and `config`

In [6]:
!cat ./properties/config.yaml

RANDOM_STATE: 0

TARGET: duration
TARGET_MIN: 1
TARGET_MAX: 60

TRAIN_SAMPLE: "green_tripdata_2021-01.parquet"
VALID_SAMPLE: "green_tripdata_2021-02.parquet"
MODEL_SAMPLE: "model_pipe.pkl"

FEATURES:
  - PULocationID
  - DOLocationID
  - trip_distance

CAT_FEATURES:
  - PULocationID
  - DOLocationID

NUM_FEATURES:
  - trip_distance


In [7]:
!pygmentize ./config/core.py

from pathlib import Path

from pydantic import BaseModel
from strictyaml import load

# Project directories
PACKAGE_ROOT = Path(__file__).resolve().parents[1]
ROOT = PACKAGE_ROOT.parent
CONFIG_FILE_PATH = PACKAGE_ROOT / "properties" / "config.yaml"
DATASET_DIR = PACKAGE_ROOT / "data"
MODEL_DIR = PACKAGE_ROOT / "models"


# Load core config object
class Config(BaseModel):
    TARGET: str
    TARGET_MIN: int
    TARGET_MAX: int
    RANDOM_STATE: int
    FEATURES: list[str]
    NUM_FEATURES: list[str]
    CAT_FEATURES: list[str]
    TRAIN_SAMPLE: str
    VALID_SAMPLE: str
    MODEL_SAMPLE: str

with open(CONFIG_FILE_PATH, "r") as f:
    config_file = f.read()

config = Config(**load(config_file).data)


In [8]:
!pygmentize ./utils/df_util.py

import seaborn as sns
import matplotlib.pyplot as plt

from config import config

def create_target_column(df):
    "Create target column"
    df[config.TARGET] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df[config.TARGET] = df[config.TARGET].dt.total_seconds() / 60
    return df

def filter_ride_duration(df):
    """Filter rides with target outliers"""
    mask = (df[config.TARGET] >= config.TARGET_MIN) & (df[config.TARGET] <= config.TARGET_MAX)
    return df[mask]

def plot_duration_histograms(y_train, yp_train, y_valid, yp_valid):
    """Plot true and prediction distributions of ride duration."""

    # fmt: off
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    pred_config = dict(label='pred', color='C0', stat='density', kde='True')
    true_config = dict(label='true', color='C1', stat='density', kde='True')
    
    ax[0].set_title("train")
    sns.histplot(yp_train, ax=ax[0], **pred_config)
    sns.histplot(y_train, ax=ax[0], **true_config)

    ax[1].set_title("va

In [9]:
!pygmentize ./utils/df_process.py

import pandas as pd

from df_util import create_target_column, filter_ride_duration
from config import config

def preprocess(data: pd.DataFrame, target: bool, filter_target: bool = False):
    """Process and clean data for feature transofrmation"""

    data[config.CAT_FEATURES] = data[config.CAT_FEATURES].astype(str)
    data[config.NUM_FEATURES] = data[config.NUM_FEATURES].astype(float)

    if target:
        data = create_target_column(data)
        if filter_target:
            data = filter_ride_duration(data)

        X = data[config.FEATURES]
        Y = data[config.TARGET].values

        return X, Y
    else:
        X = data[config.FEATURES]
        return X


The following will show the core classes used in this experiment project, which shall be the following list:

- `feature_util.py`         - contains feature selection functions
- `model_lienar.py`         - contains a linear model with mlflow integration (logging)
- `model_linear_pudo.py`    - contains a linear model using the pu_do feature from `feature_util`
- `model_trees.py`          - contains tree based models while using autolog feature of mlflow

In [10]:
!pygmentize ./utils/feature_util.py

import time
from pathlib import Path

import mlflow
import pandas as pd
from toolz import compose

from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer

from df_util import plot_duration_histograms
from config import config

ROOT_DIR = Path(__file__).parents[0]
DATA_DIR = ROOT_DIR / "data"
EXPERIMENT_NAME = "nyc-green-taxi"
TRACKING_URI = "http://127.0.0.1:5001"

def setup_experiment():
    mlflow.set_tracking_uri(TRACKING_URI)
    mlflow.set_experiment(EXPERIMENT_NAME)

def data_dict(debug=False):
    train_data_path = DATA_DIR / "green_tripdata_2021-01.parquet"
    valid_data_path = DATA_DIR / "green_tripdata_2021-02.parquet"
    train_data = pd.read_parquet(train_data_path)
    valid_data = pd.read_parquet(valid_data_path)

    return {
        "train_data": train_data if not debug else train_data[:100],
        "valid_data": valid_data if 

In [12]:
!pygmentize model_linear.py

import os

import mlflow
from sklearn.linear_model import LinearRegression

from utils.df_process import preprocess
from utils.feature_util import (
    data_dict,
    feature_pipeline,
    setup_experiment,
    mlflow_default_logging,
)

setup_experiment()
data = data_dict(debug=int(os.environ["DEBUG"]))

with mlflow.start_run():
    # Preprocessing
    X_train, y_train = preprocess(data["train_data"], target=True, filter_target=True)
    X_valid, y_valid = preprocess(data["valid_data"], target=True, filter_target=True)

    # Fit feature pipe
    feature_pipe = feature_pipeline()
    X_train = feature_pipe.fit_transform(X_train)
    X_valid = feature_pipe.transform(X_valid)

    # Fit model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # MLflow logging
    MODEL_TAG = "linear"
    mlflow_default_logging(model, MODEL_TAG, data, X_train, y_train, X_valid, y_valid)


In [13]:
!pygmentize model_linear_pudo.py

import os

import mlflow
from sklearn.linear_model import LinearRegression

from utils.df_process import preprocess
from utils.feature_util import (
    data_dict,
    feature_pipeline,
    setup_experiment,
    mlflow_default_logging,
    add_pudo_column,
    feature_selector,
)

setup_experiment()
data = data_dict(debug=int(os.environ["DEBUG"]))

with mlflow.start_run():
    # Preprocessing
    X_train, y_train = preprocess(data["train_data"], target=True, filter_target=True)
    X_valid, y_valid = preprocess(data["valid_data"], target=True, filter_target=True)

    # Feature engineering + selection
    transforms = [add_pudo_column, feature_selector]

    # Fit feature pipe
    feature_pipe = feature_pipeline(transforms)
    X_train = feature_pipe.fit_transform(X_train)
    X_valid = feature_pipe.transform(X_valid)

    # Fit model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # MLflow logging
    MODEL_TAG = "linear_pudo"
    mlflow_default_logging(model, MODE

In [14]:
!pygmentize model_trees.py

import os

import mlflow
from sklearn.ensemble import (
    ExtraTreesRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
)

from utils.df_process import preprocess
from utils.feature_util import (
    data_dict,
    feature_pipeline,
    setup_experiment,
    mlflow_default_logging,
)

setup_experiment()
mlflow.sklearn.autolog()
data = data_dict(debug=int(os.environ["DEBUG"]))

def run(model_class):
    with mlflow.start_run():
        # Preprocessing
        train = data["train_data"]
        valid = data["valid_data"]
        X_train, y_train = preprocess(train, target=True, filter_target=True)
        X_valid, y_valid = preprocess(valid, target=True, filter_target=True)

        # Fit feature pipe
        feature_pipe = feature_pipeline()
        X_train = feature_pipe.fit_transform(X_train)
        X_valid = feature_pipe.transform(X_valid)

        # Fit model
        model = model_class()
        model.fit(X_train, y_train)

        # MLFLow logging (default + fe